In [ ]:
# Data Manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.dates as mdates

# Machine Learning / Time Series
from scipy import stats
import statsmodels.api as sm
from factor_analyzer import FactorAnalyzer

# Snowflake & ETC
import datetime
import copy

In [ ]:
df = pd.read_csv("dog_data_clean.csv", index_col= [0])
df.head(2)

In [ ]:
df = df.drop('temp', axis = 1)

In [ ]:
def remove_dup_set(input_list):
    result = []
    for ele in input_list:
        if ele not in result:
            result.append(ele)
    return result

In [ ]:
corr_df = df.corr()

strong_neg_list = []
weak_neg_list = []
strong_pos_list = []
weak_pos_list = []

for col in corr_df.columns:
    strong_neg = np.where(corr_df[col] < -0.7)
    weak_neg = np.where((corr_df[col] >= -0.7) & (corr_df[col] < -0.4))
    strong_pos = np.where((corr_df[col] > 0.7) & (corr_df[col] != 1))
    weak_pos = np.where((corr_df[col] <= 0.7) & (corr_df[col] > 0.4))

    strong_neg_list_temp = list(corr_df.index[strong_neg])
    weak_neg_list_temp = list(corr_df.index[weak_neg])
    strong_pos_list_temp = list(corr_df.index[strong_pos])
    weak_pos_list_temp = list(corr_df.index[weak_pos])

    strong_neg_list_temp = [{ele,col} for ele in strong_neg_list_temp]
    weak_neg_list_temp = [ {ele,col} for ele in weak_neg_list_temp]
    strong_pos_list_temp = [{ele,col} for ele in strong_pos_list_temp]
    weak_pos_list_temp = [{ele,col} for ele in weak_pos_list_temp]

    strong_neg_list = strong_neg_list + strong_neg_list_temp
    weak_neg_list = weak_neg_list + weak_neg_list_temp
    strong_pos_list = strong_pos_list + strong_pos_list_temp
    weak_pos_list = weak_pos_list + weak_pos_list_temp

strong_neg_list  = remove_dup_set(strong_neg_list)
weak_neg_list = remove_dup_set(weak_neg_list)
strong_pos_list = remove_dup_set(strong_pos_list)
weak_pos_list = remove_dup_set(weak_pos_list)

In [ ]:
# Let's focus on the one that are hightly correlated

# what can we observe from here
sns.set(font_scale=1.7)
fig, ax = plt.subplots(figsize=(20,20))  
sns.heatmap(df.corr(), ax=ax, annot = True, vmin = 0.4, vmax = 1)

high 0.7 ~
moderate 0.4 ~ 0.7
weak ~0.4

There are high correlation between weight and height
moderate correlation between
- weight & Doorling
- height & Drooling
- open to stranger & playfulness
- energy level & playfulness
- mental stimulation needs & playfullness
- mental stimulation needs & energy level

# Let's focus on the one that are hightly correlated

# what can we observe from here


In [ ]:
# what can we observe from here
sns.set(font_scale=1.7)
fig, ax = plt.subplots(figsize=(20,20))  
sns.heatmap(df.corr(), ax=ax, annot = True, vmin = -1, vmax = -0.4)

In [ ]:
df_factor = df.drop(['dog','color','marking','health','grooming','excercise','training','nutrition','Coat Type','Coat Length'],axis = 1)

In [ ]:
df_factor.info()

Factor Analysis

In [ ]:
# Adequacy Test
# Before you perform factor analysis, you need to evaluate the “factorability” of our dataset.
# Factorability means "can we found the factors in the dataset?". There are two methods to check the factorability or sampling adequacy:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value,p_value=calculate_bartlett_sphericity(df_factor)
chi_square_value, p_value

from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(df_factor)
kmo_model

# Create factor analysis object and perform factor analysis
fa = FactorAnalyzer()
fa.fit(df_factor)
ev, v = fa.get_eigenvalues()

plt.scatter(np.arange(1,len(ev)+1),ev)
plt.plot(np.arange(1,len(ev)+1),ev)
plt.title('Scree Plot')
plt.xlabel('factors')
plt.ylabel('eigenvalues of the factors')

fa = FactorAnalyzer(3, rotation="varimax")
fa.fit(df_factor)
fa.loadings_
dir(fa)

fig, ax = plt.subplots(figsize=(10,10))         # Sample figsize in inches
sns.heatmap(fa.loadings_, annot = True, yticklabels= df_factor.columns, cmap="rocket_r", ax = ax)

# df_summary = pd.DataFrame(fa.get_factor_variance())
# df_summary.columns = ['Factor1','Factor2','Factor3']
# df_summary.index = ['SS Loadings', 'Proportion Var','Cumulative Var']

# df_summary

# from sklearn.preprocessing import StandardScaler
# cols = df.columns.drop(['DATE','ECOMM_SALES_AMT', 'RETAIL_SALES_AMT'])
# # Separating out the features
# x = df[cols].values
# # Standardizing the features
# x = StandardScaler().fit_transform(x)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=2)
# principalComponents = pca.fit_transform(x)
# principalDf = pd.DataFrame(data = principalComponents
#              , columns = ['principal component 1', 'principal component 2'])
# ecomm_series = df[['ECOMM_SALES_AMT']].reset_index(drop=True)
# finalDf = pd.concat([principalDf, ecomm_series], axis = 1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import FactorAnalysis, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris


# Load data
data = df[['DATE'] + ecomm_variables]
data = lag_df(data,ecomm_lag_dict)
data = data.drop(['DATE'] + traffic_variables, axis = 1)
feature_names = data.columns
X = StandardScaler().fit_transform(data)

# %%
# Run factor analysis with Varimax rotation
n_comps = 8

methods = [
    ("PCA", PCA()),
    ("Unrotated FA", FactorAnalysis()),
    ("Varimax FA", FactorAnalysis(rotation="varimax")),
]
fig, axes = plt.subplots(ncols=len(methods), figsize=(10, 8))
for ax, (method, fa) in  zip(axes,methods):
    fa.set_params(n_components=n_comps)
    fa.fit(X)

    fa_loadings = fa.components_.T
    # variance explained
    total_var = X.var(axis=0).sum()  # total variance of original variables,
                                    # equal to no. of vars if they are standardized
    var_exp = np.sum(fa_loadings**2, axis=0)
    prop_var_exp = var_exp/total_var
    cum_prop_var_exp = np.cumsum(var_exp/total_var)
    print(f"variance explained: {var_exp.round(3)}")
    print(f"proportion of variance explained: {prop_var_exp.round(3)}")
    print(f"cumulative proportion of variance explained: {cum_prop_var_exp.round(3)}")
    ax.scatter(x = list(range(len(cum_prop_var_exp))) ,y = prop_var_exp)

x= [1,2]
y = [100,200]
plt.scatter(x=x,y=y)

# %%
# Run factor analysis with Varimax rotation

n_comps = 3

methods = [
    ("PCA", PCA()),
    ("Unrotated FA", FactorAnalysis())
    ,("Varimax FA", FactorAnalysis(rotation="varimax")),
]
fig, axes = plt.subplots(ncols=len(methods), figsize=(20, 8))
i = 0
for ax, (method, fa) in zip(axes, methods):
    i += 1
    fa.set_params(n_components=n_comps)
    fa.fit(X)

    components = fa.components_.T
    print("\n\n %s :\n" % method)
    print(components)
    vmax = np.abs(components).max()
    if i == 1:
        sns.heatmap(components, ax= ax, vmax = vmax, vmin= -vmax, cmap="RdBu_r", annot= True, yticklabels=feature_names)
    else:
        sns.heatmap(components, ax= ax, vmax = vmax, vmin= -vmax, cmap="RdBu_r", annot= True, yticklabels=[])



    ax.set_title(str(method))
fig.suptitle("Factors")
plt.tight_layout()
plt.show()
# %%

# Run factor analysis with Varimax rotation

n_comps = 3

methods = [
    ("Unrotated FA", FactorAnalysis())
]
fig, axes = plt.subplots(ncols=len(methods), figsize=(20, 8))
i = 0
for (method, fa) in methods:
    i += 1
    fa.set_params(n_components=n_comps)
    fa.fit(X)

    components = fa.components_.T
    print("\n\n %s :\n" % method)
    print(components)
    vmax = np.abs(components).max()
    sns.set(font_scale=2)
    sns.heatmap(components, vmax = vmax, vmin= -vmax, cmap="RdBu_r", annot= True, yticklabels=feature_names,xticklabels = [1,2,3])



    ax.set_title(str(method))
fig.suptitle("Factors")
plt.tight_layout()
plt.show()

fa.score(X)
m = fa.components_
n = fa.noise_variance_
m1 = m**2
m2 = np.sum(m1,axis=1)
pvar1 = (100*m2[0])/np.sum(m2)
pvar2 = (100*m2[1])/np.sum(m2)
pvar3 = (100*m2[2])/np.sum(m2)
print(pvar1,pvar2,pvar3)
print(pvar1 + pvar2 + pvar3)

fa_loadings = fa.components_.T    # loadings

# variance explained
total_var = X.var(axis=0).sum()  # total variance of original variables,
                                 # equal to no. of vars if they are standardized

var_exp = np.sum(fa_loadings**2, axis=0)
prop_var_exp = var_exp/total_var
cum_prop_var_exp = np.cumsum(var_exp/total_var)

print(f"variance explained: {var_exp.round(3)}")
print(f"proportion of variance explained: {prop_var_exp.round(3)}")
print(f"cumulative proportion of variance explained: {cum_prop_var_exp.round(3)}")